In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

Window = None
Shader_programm = None
Vao = None
WIDTH = 800
HEIGHT = 600

In [2]:
def redimensionaCallback(window, w, h):
    global WIDTH, HEIGHT
    WIDTH = w
    HEIGHT = h

In [3]:
def inicializaOpenGL():
    global Window, WIDTH, HEIGHT

    #Inicializa GLFW
    glfw.init()

    #Criação de uma janela
    Window = glfw.create_window(WIDTH, HEIGHT, "Exemplo - renderização de um triângulo", None, None)
    if not Window:
        glfw.terminate()
        exit()

    glfw.set_window_size_callback(Window, redimensionaCallback)
    glfw.make_context_current(Window)

    print("Placa de vídeo: ",OpenGL.GL.glGetString(OpenGL.GL.GL_RENDERER))
    print("Versão do OpenGL: ",OpenGL.GL.glGetString(OpenGL.GL.GL_VERSION))

In [4]:
def inicializaObjetos():

    global Vao
    # Vao do quadrado
    Vao = glGenVertexArrays(1)
    glBindVertexArray(Vao)

    # VBO dos vértices do quadrado
    points = [
        # triângulo 1
		0.5, 0.5, 0.0, #vertice superior direito
		0.5, -0.5, 0.0, #vertice inferior direito
		-0.5, -0.5, 0.0, #vertice inferior esquerdo
		#triângulo 2
		-0.5, 0.5, 0.0, #vertice superior esquerdo
		0.5, 0.5, 0.0, #vertice superior direito
		-0.5, -0.5, 0.0 #vertice inferior esquerdo
	]
    points = np.array(points, dtype=np.float32)
    pvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, pvbo)
    glBufferData(GL_ARRAY_BUFFER, points, GL_STATIC_DRAW)
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 0, None)

    # VBO das cores
    cores = [
		#triângulo 1
		1.0, 1.0, 0.0,#amarelo
		0.0, 1.0, 1.0,#ciano
		1.0, 0.0, 1.0,#magenta
		#triângulo 2
		0.0, 1.0, 1.0,#ciano
		1.0, 1.0, 0.0,#amarelo
		1.0, 0.0, 1.0,#magenta
	]
    cores = np.array(cores, dtype=np.float32)
    cvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, cvbo)
    glBufferData(GL_ARRAY_BUFFER, cores, GL_STATIC_DRAW)
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 0, None)

In [5]:
def inicializaShaders():
    global Shader_programm
    global Shader_2
    # Especificação do Vertex Shader:
    vertex_shader = """
        #version 400
        layout(location = 0) in vec3 vertex_posicao; //vem da modelagem de um objeto no python
        layout(location = 1) in vec3 vertex_cores; //vem da modelagem de um objeto no python
        uniform mat4 matriz; //matriz 4x4 vem do python, pois ela tem o modificador "uniform"
        out vec3 cores;
        void main () {
            cores = vertex_cores;
            gl_Position = matriz * vec4 (vertex_posicao, 1.0);
        }
    """
    vs = OpenGL.GL.shaders.compileShader(vertex_shader, GL_VERTEX_SHADER)
    if not glGetShaderiv(vs, GL_COMPILE_STATUS):
        infoLog = glGetShaderInfoLog(vs, 512, None)
        print("Erro no vertex shader:\n", infoLog)

    # Especificação do Fragment Shader:
    fragment_shader = """
        #version 400
        in vec3 cores;
		out vec4 frag_colour;
		void main () {
		    frag_colour = vec4 (cores, 1.0);
		}
    """
    fs = OpenGL.GL.shaders.compileShader(fragment_shader, GL_FRAGMENT_SHADER)
    if not glGetShaderiv(fs, GL_COMPILE_STATUS):
        infoLog = glGetShaderInfoLog(fs, 512, None)
        print("Erro no fragment shader:\n", infoLog)
        
    fragment_shader_2 = """
        #version 400
        in vec3 cores;
		out vec4 frag_colour;
		void main () {
		    frag_colour = vec4 (1-cores, 1.0);
		}
    """
    fs2 = OpenGL.GL.shaders.compileShader(fragment_shader_2, GL_FRAGMENT_SHADER)
    if not glGetShaderiv(fs2, GL_COMPILE_STATUS):
        infoLog = glGetShaderInfoLog(fs2, 512, None)
        print("Erro no fragment shader 2:\n", infoLog)

    # Especificação do Shader Programm:
    Shader_programm = OpenGL.GL.shaders.compileProgram(vs, fs)
    if not glGetProgramiv(Shader_programm, GL_LINK_STATUS):
        infoLog = glGetProgramInfoLog(Shader_programm, 512, None)
        print("Erro na linkagem do shader:\n", infoLog)
        
    Shader_2 = OpenGL.GL.shaders.compileProgram(vs, fs2)
    if not glGetProgramiv(Shader_2, GL_LINK_STATUS):
        infoLog = glGetProgramInfoLog(Shader_2, 512, None)
        print("Erro na linkagem do shader 2:\n", infoLog)

    glDeleteShader(vs)
    glDeleteShader(fs)
    glDeleteShader(fs2)

In [6]:
def transformaQuadrado(matriz):
    #matriz de translação
    
    #E passamos a matriz para o Vertex Shader.
    #descobre o endereço de memória (de vídeo) da variável matriz lá no shader
    transformLoc = glGetUniformLocation(Shader_programm, "matriz") 
    #passa os valores da matriz aqui do python para a memória de vídeo no endereço descoberto acima
    glUniformMatrix4fv(transformLoc, 1, GL_TRUE, matriz)

In [7]:
def translacao(matriz, x, y):
        target = np.array([[1,0,0,x],
                           [0,1,0,y],
                           [0,0,1,0],
                           [0,0,0,1]], np.float32)
        return target.dot(matriz)

In [8]:
def escala(matriz, x, y):
        target = np.array([[x,0,0,0],
                           [0,y,0,0],
                           [0,0,1,0],
                           [0,0,0,1]], np.float32)
        return target.dot(matriz)

In [9]:
def rotacao(matriz, a):
        angulo = np.radians(a)
        cos = np.cos(angulo)
        sen = np.sin(angulo)
        target = np.array([[cos,-sen,0,0],
                           [sen,cos,0,0],
                           [0,0,1,0],
                           [0,0,0,1]], np.float32)
        return target.dot(matriz)

In [10]:
def inicializaRenderizacao():
    global Window, Shader_programm, Vao, WIDTH, HEIGHT
    
    matriz = np.array([
        [1.0, 0.0, 0.0, 0.0],
        [0.0, 1.0, 0.0, 0.0],
        [0.0, 0.0, 1.0, 0.0], 
        [0.0, 0.0, 0.0, 1.0]], np.float32)
    
    opt = True

    while not glfw.window_should_close(Window):
        glClear(GL_COLOR_BUFFER_BIT)
        glClearColor(0.2, 0.3, 0.3, 1.0)
        glViewport(0, 0, WIDTH, HEIGHT)
        
        if(opt == True):
            glUseProgram(Shader_programm)
        else:
            glUseProgram(Shader_2)
        
        if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_SPACE)):
            if(opt == True):
                opt = False
            else:
                opt = True

        glBindVertexArray(Vao) #ativa o objeto a ser renderizado
        
        transformaQuadrado(matriz)
        
        # MUDEI OS BOTOES DOS ESPECIFICADOS NA ATIVIDADE
        # W A S D - Translação
        # I K - Escala
        # J L - Rotação
        # Espaço - Shader
        
        if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_W)):
            matriz = translacao(matriz, 0, 0.001)
            
        if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_S)):
            matriz = translacao(matriz, 0, -0.001)
            
        if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_D)):
            matriz = translacao(matriz, 0.001, 0)
            
        if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_A)):
            matriz = translacao(matriz, -0.001, 0)
            
        if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_L)):
            matriz = rotacao(matriz, -0.1)
            
        if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_J)):
            matriz = rotacao(matriz, 0.1)
            
        if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_I)):
            matriz = escala(matriz, 1.001, 1.001)
            
        if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_K)):
            matriz = escala(matriz, 0.999, 0.999)

        glDrawArrays(GL_TRIANGLES, 0, 6) #renderiza o objeto

        glfw.poll_events()

        glfw.swap_buffers(Window)
        
        if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_ESCAPE)):
            glfw.set_window_should_close(Window, True)
    
    glfw.terminate()

In [11]:
def main():
    inicializaOpenGL()
    inicializaObjetos()
    inicializaShaders()
    inicializaRenderizacao()

In [12]:
main()

Placa de vídeo:  b'Intel(R) HD Graphics 630'
Versão do OpenGL:  b'4.6.0 - Build 26.20.100.8142'
